In [1]:
import os

In [2]:
%pwd

'd:\\Sagar_office\\repo\\chicken-disease-classification-Project\\research'

In [3]:
from pathlib import Path
os.chdir(Path("D:/Sagar_office/repo/chicken-disease-classification-Project"))

In [4]:
%pwd

'D:\\Sagar_office\\repo\\chicken-disease-classification-Project'

In [5]:
# entity
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataIngestionConfig:
    root_dir : Path
    source_URL: Path
    local_data_file: Path
    unzip_dir: Path

In [6]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml, create_directories

In [7]:
# config manager
class ConfigurationManager:
    def __init__(
            self,
            config_filepath = CONFIG_FILE_PATH,
            params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion
        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL = config.source_URL,
            local_data_file = config.local_data_file,
            unzip_dir = config.unzip_dir
        )
        return data_ingestion_config

In [8]:
import os
import urllib.request as request
import zipfile
from src.cnnClassifier import logger
from src.cnnClassifier.utils.common import get_size

In [9]:
# components
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config

    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            fileName, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{fileName} downloaded with following info: \n{headers}")
        else:
            logger.info(f"file already exists of size: {get_size(Path(self.config.local_data_file))}")

    def extract_zip_file(self):
        """
        zip_file_path : str
        Extracts the zip file in data directory
        Function returns None
        """

        unzip_path = self.config.unzip_dir
        print(unzip_path)
        os.makedirs(unzip_path, exist_ok = True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [11]:
# create pipelines
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config = data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
   raise e

[2024-10-30 05:46:09,417: INFO: yaml file: config\config.yaml loaded successfully]
[2024-10-30 05:46:09,424: INFO: yaml file: params.yaml loaded successfully]
[2024-10-30 05:46:09,426: INFO: create directory at: artifacts]
[2024-10-30 05:46:09,428: INFO: create directory at: artifacts/data_ingestion]
[2024-10-30 05:46:15,110: INFO: artifacts/data_ingestion/data.zip downloaded with following info: 
Connection: close
Content-Length: 11605269
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "ca3b15509c98d4859f3a277940e07db6021a9f4c4bc3f28947c392bcebddc7e7"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 0F5F:12C87C:4DB909:6208A1:67217AC5
Accept-Ranges: bytes
Date: Wed, 30 Oct 2024 00:16:11 GMT
Via: 1.1 varnish
X-Served-By: cache-bom4721-BOM
X-Cache: MISS
X-Cache-Hits: 0
X-Timer: S1730247370.1038